In [ ]:
# A Python dictionary...
mydict = {'name': 'Joe Doe',
          'age': 23,
          'married': False,
          'children': None,
          'hobbies': ['hiking', 'reading']}

In [ ]:
# ...converted to a JSON string
# by json.dumps. The "indent" parameter is
# optional and prettyfies the printing.
import json
myjson = json.dumps(mydict, indent=4)
print(myjson)

In [ ]:
# Convert the JSON string back to a native Python data structure
json.loads(myjson)

In [ ]:
import requests
response = requests.get("https://pypi.org/pypi/pandas/json")
response.status_code

In [ ]:
# response.json()

In [ ]:
releases = []
for version, files in response.json()['releases'].items():
    releases.append(f"{version}: {files[0]['upload_time']}")
print(releases[:3])  # # print the first 3 elements of the list

In [ ]:
import urllib.parse
urllib.parse.quote_plus("pa$$word")

In [ ]:
# Our SQL query
sql = "SELECT * FROM packages"

In [ ]:
# Option 1: Database driver (sqlite3 is part of the standard library)
import sqlite3
# Using the connection as context manager automatically commits
# the transaction or rolls it back in case of an error
with sqlite3.connect('packagetracker/packagetracker.db') as con:
    cursor = con.cursor() # We need a cursor to run SQL queries
    result = cursor.execute(sql).fetchall()  # Return all records
result

In [ ]:
# Option 2: SQLAlchemy
from sqlalchemy import create_engine
# create_engine expects the connection string of your database
engine = create_engine('sqlite:///packagetracker/packagetracker.db')
with engine.connect() as con:
    result = con.execute(sql).fetchall()
result

In [ ]:
# Option 3: pandas
# "read_sql_table" reads the full table
# Pandas requires an SQLAlchemy engine that we reuse from
# the previous example
import pandas as pd
df = pd.read_sql_table('packages', engine, index_col='package_id')
df

In [ ]:
# "read_sql_query" runs an SQL query
pd.read_sql_query(sql, engine, index_col='package_id')

In [ ]:
# "to_sql" writes DataFrames to tables
# "if_exists" has to be either "fail", "append" or "replace"
# and defines what happens if the table already exists
df.to_sql('packages2', con=engine, if_exists='append')

In [ ]:
# The previous command creates a new table and inserts the records
# from the DataFrame df as we can verify by reading it back
pd.read_sql_table('packages2', engine, index_col='package_id')

In [ ]:
# Let's get rid of the table again by using the
# "drop table" command and running it via SQLAlchemy
with engine.connect() as con:
    con.execute("DROP TABLE packages2")

In [ ]:
sql = """
SELECT v.uploaded_at, v.version_string
FROM packages p
INNER JOIN package_versions v ON p.package_id = v.package_id
WHERE p.package_id = :package_id
ORDER BY v.uploaded_at
"""

In [ ]:
# Via SQLAlchemy
from sqlalchemy.sql import text
with engine.connect() as con:
    result = con.execute(text(sql), package_id=1).fetchall()
result[:3]  # print the first 3 records

In [ ]:
# Via pandas
pd.read_sql_query(text(sql), engine, parse_dates=['uploaded_at'],
                  params={'package_id': 1},
                  index_col=['uploaded_at']).head(3)